Tutorial - Part #3 - Advanced SingleImage
=======================

In this tutorial a more advanced set of examples are presented on SingleImage class,
which allows tod do more specific tasks with the instances.

We import the packages, and also a pair of sample images

In [1]:
import properimage.single_image as si